# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
# Write your answer here

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

**1.** We start by importing the data from the file.

In [ ]:
# Write your answer here
import pandas as pd
titanic = pd.read_excel(DATA_FOLDER + '/titanic.xls', sheetname='titanic')
titanic

Next we can list the data types of each field.

In [ ]:
titanic.dtypes

When it comes to the `object` fields, we can be a bit more precise. `name`, `sec`, `ticket`, `cabin`, `embarked`, `boat` and `home.dex` are all strings.

Next, we call the `describe` method to list some statistics on the data. We thus obtain the range of all of the numeric fields of our data.

In [ ]:
titanic.describe()

Moreover, we can also note some ranges of other fields. For example, `sex` has only two possible values `female` and `male`. `embarked` can only be `S`, `C` and `Q`.

For a better visual result, we decided to replace the travel classes, ports to more readable values. As we make  them categorical types, the performance stays the same. 

In [ ]:
titanic['pclass'] = titanic['pclass'].apply(lambda x: ("First" if x == 1 else ("Second" if x == 2 else "Third")) + " Class")

titanic['survived'] = titanic['survived'].apply(lambda x: "Survived" if x == 1 else "Deceased")

titanic['embarked'] = titanic['embarked'].apply(lambda x: "Cherbourg" if x == 'C' else ("Queenstown" if x == 'Q' else "Southampton"))

Then we make categorical data as actually categorical.

In [ ]:
titanic['pclass'] = titanic.pclass.astype('category')
titanic['survived'] = titanic.survived.astype('category')
titanic['sex'] = titanic.sex.astype('category')
titanic['embarked'] = titanic.embarked.astype('category')
titanic['cabin'] = titanic.cabin.astype('category')
titanic['boat'] = titanic.boat.astype('category')

**2.** We plot the histogram of the travel class.

In [ ]:
titanic.pclass.value_counts(sort=False).plot(kind='bar')

Next we plot the histogram of the three embark ports.

In [ ]:
titanic.embarked.value_counts().plot(kind='bar')

Next we plot the histogram of the sex.

In [ ]:
titanic.sex.value_counts().plot(kind='bar')

Next, we cut the ages data into decades and plot the histogram of the devades.

In [ ]:
pd.cut(titanic.age, range(0,90,10)).value_counts(sort=False).plot(kind='bar')

**3.** We plot the cabin floor data as a pie chart.

In [ ]:
titanic.cabin.dropna().apply(lambda x : x[0]).value_counts(sort=False).plot(kind='pie')

**4.** Here, we plot the proportion of people that survived in the first class.

In [ ]:
titanic[titanic.pclass == "First Class"].survived.value_counts(sort=False).plot(kind='pie')

Next, we plot the proportion of people that survived in the second class.

In [ ]:
titanic[titanic.pclass == "Second Class"].survived.value_counts(sort=False).plot(kind='pie')

Finally, we plot the proportion of people that survived in the third class.

In [ ]:
titanic[titanic.pclass == "Third Class"].survived.value_counts(sort=False).plot(kind='pie')

**5.** Here we add new columns that will help us to calculate proportions of survived people in the last part.

In [ ]:
titanic.insert(0, 'alive', 0)
titanic.insert(0, 'dead', 0)
titanic.insert(0, 'ratio', 0)

Here we set these new columns to appropriate values. We essentialy separate the survived columns for easier summing later on. Finnaly we slice the data to take only the columns of interest.

In [ ]:
titanic.loc[titanic['survived'] == "Survived", 'alive'] = 1
titanic.loc[titanic['survived'] == "Deceased", 'dead'] = 1
df = titanic[['pclass', 'sex', 'alive', 'dead', 'ratio']]

We group the data by the sec and class of the passangers and we sum it. Then we have the sum of alive and dead people groupped as we wish and we can easily calculate the proportion of them that survived, which we plot as a histogram.

In [ ]:
aggregated = df.groupby(['sex', 'pclass']).sum()
(aggregated['alive'] / (aggregated['alive'] + aggregated['dead'])).plot(kind='bar')

**6.** Next we insert a new column that will be the age category of each person. Since we wan't to split the people in two equal groups based on age, we compute the median age of passangers. We also drop the passengers with an unknown age value, to avoid bad results for the median computation. 

In [ ]:
titanic.dropna(axis=0, subset=['age'], inplace=True)
titanic.insert(0, 'age_category', 0)
median = titanic['age'].median()

Next, we set the correct category to people below or above the median age. The people that have the median age are grouped with the people below it. Next we set this column as a categorical column.

In [ ]:
titanic.loc[titanic['age'] > median, 'age_category'] = "Age > " + str(median)
titanic.loc[titanic['age'] <= median, 'age_category'] = "Age <= " + str(median)
titanic['age_category'] = titanic.age_category.astype('category')

Next, we take the columns that are of interest to us and group by age category, sec and travel class. Then we sum over these groups, obtaining the people that lived and those that died which which we can compute the proportion and display it as a dataframe.

In [ ]:
sub = titanic[['pclass', 'sex', 'age_category', 'alive', 'dead', 'ratio']]
subagg = sub.groupby(['age_category', 'sex', 'pclass']).sum()
subagg['ratio'] = (subagg['alive'] / (subagg['alive'] + subagg['dead']))
only_ratio = subagg[['ratio']]
only_ratio